In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/02/19 10:02:54 WARN Utils: Your hostname, isolde resolves to a loopback address: 127.0.1.1; using 192.168.0.188 instead (on interface wlp1s0)
23/02/19 10:02:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/19 10:02:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/19 10:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/19 10:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/02/19 10:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/02/19 10:02:56 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

/opt/spark-3.3.1-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [5]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [6]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

23/02/19 10:11:16 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


23/02/19 10:11:16 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [7]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [17]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [18]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

23/02/19 10:33:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


23/02/19 10:33:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [27]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [28]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [29]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [30]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [31]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  22|              15.8|                   1|              null|                 null|
|2020-01-01 00:00:00|  25|             531.0|                  26|            324.35|                   16|
|2020-01-01 00:00:00|  55|129.29000000000002|                   4|              null|                 null|
|2020-01-01 00:00:00|  56|             99.69|                   3|              18.1|                    2|
|2020-01-01 00:00:00|  60|            160.04|                   6|57.620000000000005|                    2|
|2020-01-01 00:00:00|  61|            526.71|                  17|            146.64|                    3|
|2020-01-01 00:00:00|  65|  

In [32]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

23/02/19 10:42:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [23]:
df_join = spark.read.parquet('data/report/revenue/total')

In [34]:
df_zones = spark.read.parquet('data/zones/')

In [35]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [37]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  22|              15.8|                   1|              null|                 null|        22| Brooklyn|    Bensonhurst West|   Boro Zone|
|2020-01-01 00:00:00|  25|             531.0|                  26|            324.35|                   16|        25| Brooklyn|         Boerum Hill|   Boro Zone|
|2020-01-01 00:00:00|  55|129.29000000000002|                   4|              null|                 null|        55| Brooklyn|        Coney Island|   Boro Zone|
|2020-01-01 00:00:00| 

In [36]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

23/02/19 11:14:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
